In [1]:
import gzip
import urllib.request as request
import pandas as pd
import io

In [2]:
def get_codes(dictionary):
      url_dict = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/BulkDownloadListing" + \
          "?sort=1&downfile=dic%2Fen%2F" +\
          dictionary + ".dic"
      with request.urlopen(url_dict) as resp:
                file_content = resp.readlines()
      d={}
      for i in file_content:
            if len(i) > 1:
                  row=i.decode().split('\t')
                  d[row[0]] = row[1].strip()
      return d

In [3]:
def get_df(dataset):
    """ 
    dataset: choose the dataset code from:
https://ec.europa.eu/eurostat/databrowser/explore/all/all_themes?lang=en&display=list&sort=category
    something like EXT_LT_INTROEU27_2020
List of datasets for downloding: 
https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?dir=data&sort=1&sort=2&start=a
Returtn the cleaned dataset with decoded abbreviation

    """

    dataset = dataset.lower()
    url = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/" + \
          "BulkDownloadListing?file=data%2F/" + \
          dataset + ".tsv.gz"

    with request.urlopen(url) as r:
            with gzip.GzipFile(fileobj=r) as data:
                file_content = data.read() #data read as binary from gzip
    # the approach to make the dataframe from the object with read data:
    # https://stackoverflow.com/questions/39213597/convert-text-data-from-requests-object-to-dataframe-with-pandas
    # the eurostat data contains not strict columns:first line with separator ',',the date columns - separator - '\t'
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    #devide one column 'geo\\date'
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    # use only file_content,not df,it is binary, so use decode()
    #file_content has the first row with the code we need to decode into names using function get_code
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [4]:
def clean_df(dataset_file):
    """ 
    use it if you have downloaded file
    https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&dir=dic%2Fen
    """

   
    with gzip.open(dataset_file, 'rb') as f:
        file_content = f.read()
    df = pd.read_csv(io.StringIO(file_content.decode('utf-8')),sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python",encoding='utf-8')
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    codes=file_content.decode().split('\t')[0].split('\\')[0].split(',')
    for c in codes:
        df[c].replace(get_codes(c),inplace=True)
    return df

In [7]:
get_df(dataset='NRG_CB_RW').head(44) #Supply, transformation and consumption of renewables and wastes

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Distribution losses,Primary solid biofuels,Terajoule,Albania,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Distribution losses,Primary solid biofuels,Terajoule,Austria,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Distribution losses,Primary solid biofuels,Terajoule,Bosnia and Herzegovina,NaN,0.000,0.000,0.000,0.000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Distribution losses,Primary solid biofuels,Terajoule,Belgium,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Distribution losses,Primary solid biofuels,Terajoule,Bulgaria,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,110.0,40.0,22.0,12.0,13.0,2.0,0.0,0.0,0.0
5,Distribution losses,Primary solid biofuels,Terajoule,Cyprus,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Distribution losses,Primary solid biofuels,Terajoule,Czechia,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Distribution losses,Primary solid biofuels,Terajoule,Germany (until 1990 former territory of the FRG),NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Distribution losses,Primary solid biofuels,Terajoule,Denmark,NaN,0.000,0.000,0.000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Distribution losses,Primary solid biofuels,Terajoule,Euro area - 19 countries (from 2015),NaN,29.000,28.000,30.000,1.000,0.0,...,20.0,24.0,33.0,10.0,10.0,2.0,2.0,0.0,0.0,80.0


In [6]:
get_df(dataset='NRG_CB_E')#Supply, transformation and consumption of electricity

,nrg_bal,siec,unit,geo,2021,2020,2019,2018,2017,2016,...,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990
0,Available for final consumption,Electricity,Gigawatt-hour,Albania,7030.299,6656.144,6530.243,6480.824,6136.388,5674.000,...,4367.0,2277.0,2148.0,2818.0,2084.0,1875.0,1697.0,1444.0,1341.0,1780.0
1,Available for final consumption,Electricity,Gigawatt-hour,Austria,66084.891,63577.133,66027.931,65855.941,65569.869,64527.283,...,51846.0,50699.0,49884.0,49270.0,47651.0,46538.0,45903.0,45281.0,45767.0,43577.0
2,Available for final consumption,Electricity,Gigawatt-hour,Bosnia and Herzegovina,10843.000,10240.000,11301.000,11820.000,11785.000,11494.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Available for final consumption,Electricity,Gigawatt-hour,Belgium,83660.400,80871.400,83306.400,84263.000,83809.800,83228.100,...,76050.0,75527.0,73321.0,71405.0,69829.0,67822.0,64644.0,63871.0,61783.0,59108.0
4,Available for final consumption,Electricity,Gigawatt-hour,Bulgaria,31684.820,30511.049,31119.715,31389.986,31445.466,30257.000,...,25358.0,27744.0,28524.0,32053.0,30746.0,27875.0,27890.0,27575.0,31230.0,36484.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Slovakia,397.000,387.000,316.000,404.000,418.000,334.000,...,336.0,307.0,295.0,340.0,438.0,348.0,539.0,533.0,540.0,1158.0
2318,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Türkiye,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2319,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,Ukraine,0.000,0.000,1834.900,1579.000,1574.200,1633.000,...,246.0,246.0,246.0,281.0,276.0,293.0,277.0,290.0,302.0,293.0
2320,Transformation input - electricity and heat ge...,Electricity,Gigawatt-hour,United Kingdom,NaN,NaN,2115.259,3390.866,3859.190,4014.340,...,3774.0,2594.0,2477.0,2430.0,2282.0,2051.0,1948.0,2257.0,2109.0,2626.0
